<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_chatgpt_gpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Use FLAML to Tune ChatGPT

FLAML offers a cost-effective hyperparameter optimization technique [EcoOptiGen](https://arxiv.org/abs/2303.04673) for tuning Large Language Models. Our study finds that tuning hyperparameters can significantly improve the utility of LLMs.

In this notebook, we tune OpenAI ChatGPT (both GPT-3.5 and GPT-4) models for math problem solving. We use [the MATH benchmark](https://crfm.stanford.edu/helm/latest/?group=math_chain_of_thought) for measuring mathematical problem solving on competition math problems with chain-of-thoughts style reasoning.

Related link: [Blogpost](https://microsoft.github.io/FLAML/blog/2023/04/21/LLM-tuning-math) based on this experiment.

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai,blendsearch] option:
```bash
pip install flaml[openai,blendsearch]
```

In [1]:
# %pip install flaml[openai,blendsearch] datasets

### Import the oai and tune subpackages from flaml

FLAML has provided an API for hyperparameter optimization of OpenAI ChatGPT models: `oai.ChatCompletion.tune` and to make a request with the tuned config: `oai.ChatCompletion.create`. First, we import oai from flaml:

In [2]:
from flaml import oai

### Set your API Endpoint

The [`config_list_openai_aoai`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_openai_aoai) function tries to create a list of  Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.


In [3]:
config_list = oai.config_list_openai_aoai()

The config list looks like the following:
```python
config_list = [
    {'api_key': '<your OpenAI API key here>'},  # only if OpenAI API key is found
    {
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # only if the at least one Azure OpenAI API key is found
    {
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # only if the second Azure OpenAI API key is found
]
```

You can directly override it if the above function returns an empty list, i.e., it doesn't find the keys in the specified locations.

## Load dataset

We load the competition_math dataset. The dataset contains 201 "Level 2" Algebra examples. We use a random sample of 20 examples for tuning the generation hyperparameters and the remaining for evaluation.

In [4]:
import datasets

seed = 41
data = datasets.load_dataset("competition_math")
train_data = data["train"].shuffle(seed=seed)
test_data = data["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "problem": train_data[x]["problem"],
        "solution": train_data[x]["solution"],
    }
    for x in range(len(train_data)) if train_data[x]["level"] == "Level 2" and train_data[x]["type"] == "Algebra"
][:n_tune_data]
test_data = [
    {
        "problem": test_data[x]["problem"],
        "solution": test_data[x]["solution"],
    }
    for x in range(len(test_data)) if test_data[x]["level"] == "Level 2" and test_data[x]["type"] == "Algebra"
]
print(len(tune_data), len(test_data))


Using custom data configuration default
Found cached dataset competition_math (/home/vscode/.cache/huggingface/datasets/competition_math/default/1.0.0/2a2a2995c2847186883ecd64f69be7d602b8a6f6b51950624d4dc2263f93333b)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/vscode/.cache/huggingface/datasets/competition_math/default/1.0.0/2a2a2995c2847186883ecd64f69be7d602b8a6f6b51950624d4dc2263f93333b/cache-f1cfe8228271b121.arrow
Loading cached shuffled indices for dataset at /home/vscode/.cache/huggingface/datasets/competition_math/default/1.0.0/2a2a2995c2847186883ecd64f69be7d602b8a6f6b51950624d4dc2263f93333b/cache-d155a2d38c23bd53.arrow


20 201


Check a tuning example:

In [5]:
print(tune_data[1]["problem"])

What is the slope of a line parallel to the line $2x - 4y = 9$? Express your answer as a common fraction.


Here is one example of the canonical solution:

In [6]:
print(tune_data[1]["solution"])

The slope-intercept form of the equation of a line is $y = m x + b$ where $m$ is the slope.  So if we get $y$ on the opposite side from $x$ and make it have a coefficient of 1, the slope of the line will be the coefficient of $x$.  Thus we add $4y$ to both sides and divide everything by 4 which makes the coefficient of $x$ equal to $\boxed{\frac{1}{2}}$.


## Define Success Metric

Before we start tuning, we need to define the success metric we want to optimize. For each math task, we use voting to select a response with the most common answers out of all the generated responses. If it has an equivalent answer to the canonical solution, we consider the task as successfully solved. Then we can optimize the mean success rate of a collection of tasks.

In [7]:
from flaml.autogen.math_utils import eval_math_responses

## Use the tuning data to find a good configuration


For (local) reproducibility and cost efficiency, we cache responses from OpenAI with a controllable seed.

In [8]:
oai.ChatCompletion.set_cache(seed)

This will create a disk cache in ".cache/{seed}". You can change `cache_path_root` from ".cache" to a different path in `set_cache()`. The cache for different seeds are stored separately.

### Perform tuning

The tuning will take a while to finish, depending on the optimization budget. The tuning will be performed under the specified optimization budgets.

* `inference_budget` is the target average inference budget per instance in the benchmark. For example, 0.004 means the target inference budget is 0.004 dollars, which translates to 2000 tokens (input + output combined) if the gpt-3.5-turbo model is used.
* `optimization_budget` is the total budget allowed to perform the tuning. For example, 1 means 1 dollars are allowed in total, which translates to 500K tokens for the gpt-3.5-turbo model.
* `num_sumples` is the number of different hyperparameter configurations which is allowed to try. The tuning will stop after either num_samples trials or after optimization_budget dollars spent, whichever happens first. -1 means no hard restriction in the number of trials and the actual number is decided by `optimization_budget`.

Users can specify tuning data, optimization metric, optimization mode, evaluation function, search spaces etc.. The default search space is:

```python
default_search_space = {
    "model": tune.choice([
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 2)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{prompt}",
}
```

The default search space can be overridden by users' input.
For example, the following code specifies a fixed prompt template. For hyperparameters which don't appear in users' input, the default search space will be used.

In [9]:
import logging

prompts = ["{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \\boxed{{}}."]
config, analysis = oai.ChatCompletion.tune(
    data=tune_data,  # the data for tuning
    metric="success_vote",  # the metric to optimize
    mode="max",  # the optimization mode
    eval_func=eval_math_responses,  # the evaluation function to return the success metrics
    # log_file_name="logs/math.log",  # the log file name
    inference_budget=0.02,  # the inference budget (dollar per instance)
    optimization_budget=1,  # the optimization budget (dollar in total)
    # num_samples can further limit the number of trials for different hyperparameter configurations;
    # -1 means decided by the optimization budget only
    num_samples=20,
    model="gpt-3.5-turbo",  # comment to tune both gpt-3.5-turbo and gpt-4
    prompt=prompts,  # the prompt templates to choose from
    # stop="###",  # the stop sequence
    config_list=config_list,  # the endpoint list
    # logging_level=logging.INFO,  # the logging level
)


[I 2023-05-20 22:12:09,860] A new study created in memory with name: optuna


[flaml.tune.tune: 05-20 22:12:09] {773} INFO - trial 1 config: {'model': 'gpt-3.5-turbo', 'temperature_or_top_p': {'top_p': 0.36280922847807595}, 'max_tokens': 347, 'n': 10, 'prompt': 0}
[flaml.tune.tune: 05-20 22:12:10] {197} INFO - result: {'expected_success': 0.8043947377449999, 'success': 0.85, 'success_vote': 0.7, 'voted_answer': 'We have $(-27)^{5/3} = \\left[(-27)^{1/3}\\right]^5$. Since $(-3)^3 = -27$, we have $(-27)^{1/3} = -3$. Therefore, $\\left[(-27)^{1/3}\\right]^5 = (-3)^5 = \\boxed{-243}$.', 'votes': 8.7, 'total_cost': 0.05771399999999999, 'cost': 0.05771399999999999, 'inference_cost': 0.0025029, 'training_iteration': 0, 'config': {'model': 'gpt-3.5-turbo', 'temperature_or_top_p': {'top_p': 0.36280922847807595}, 'max_tokens': 347, 'n': 10, 'prompt': 0}, 'config/model': 'gpt-3.5-turbo', 'config/temperature_or_top_p': {'top_p': 0.36280922847807595}, 'config/max_tokens': 347, 'config/n': 10, 'config/prompt': 0, 'experiment_tag': 'exp', 'time_total_s': 0.22889995574951172}
[

### Output tuning results

After the tuning, we can print out the config and the result found by FLAML:

In [10]:
print("optimized config", config)
print("best result on tuning data", analysis.best_result)

optimized config {'max_tokens': 470, 'n': 50, 'prompt': '{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \\boxed{{}}.', 'model': 'gpt-3.5-turbo', 'temperature': 1.2672964698525508}
best result on tuning data {'expected_success': 0.9999161685004145, 'success': 1.0, 'success_vote': 0.9, 'voted_answer': 'We have \\begin{align*}\n(-27)^{5/3} &= ((-27)^{1/3})^{5}\\\\\n&=(\\sqrt[3]{-27})^5 &&\\mbox{(by definition of cube root)}\\\\\n&= (-3)^5 && \\mbox{(because $(-a)^{2n+1}= -a^{2n+1}$ for all real numbers $a$ and integers $n \\geq 0$)} \\\\\n&= -243.\n\\end{align*}Thus, $(-27)^{5/3} = \\boxed{-243}$.', 'votes': 33.8, 'total_cost': 0.37417999999999996, 'cost': 0.316466, 'inference_cost': 0.0149563, 'training_iteration': 0, 'config': {'temperature_or_top_p': {'temperature': 1.2672964698525508}, 'max_tokens': 470, 'n': 50, 'prompt': 0, 'model': 'gpt-3.5-turbo'}, 'config/temperature_or_top_p': {'temperature': 1.2672964698525508}, 'config/

### Make a request with the tuned config

We can apply the tuned config on the request for an example task:

In [11]:
response = oai.ChatCompletion.create(context=tune_data[1], config_list=config_list, **config)
metric_results = eval_math_responses(oai.ChatCompletion.extract_text(response), **tune_data[1])
print("response on an example data instance:", response)
print("metric_results on the example data instance:", metric_results)


response on an example data instance: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Given $2x-4y=9$. Putting this in slope-intercept form yields $-2y = -2x + 9 \\Rightarrow y=x-\\frac{9}{2}$, so the slope is $\\boxed{1}$. Parallel lines have equal slopes.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "To find the slope of a line parallel to $2x-4y=9,$ we need to find the slope of the line $2x-4y=9.$ Solving for $y$ in this first equation, we get $$\\begin{aligned} 2x-4y&=9\n\\\\\\Rightarrow\\qquad 4y&=2x-9\n\\\\\\Rightarrow\\qquad y&=\\frac{1}{2}x-\\frac{9}{4}.\\end{aligned}$$So the slope of this line is $\\frac{1}{2}.$ Therefore, the slope of any line parallel to this line will also be $\\boxed{\\frac{1}{2}}.$",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "message": {
   

### Evaluate the success rate on the test data

You can use flaml's `oai.ChatCompletion.test` to evaluate the performance of an entire dataset with the tuned config. The following code will take a while (30 mins to 1 hour) to evaluate all the test data instances if uncommented and run. It will cost roughly $3. 

In [15]:
# result = oai.ChatCompletion.test(test_data, logging_level=logging.INFO, config_list=config_list, **config)
# print("performance on test data with the tuned config:", result)

[flaml.autogen.oai.completion: 05-20 22:21:33] {927} INFO - evaluating data instance 0
[flaml.autogen.oai.completion: 05-20 22:22:43] {927} INFO - evaluating data instance 1
[flaml.autogen.oai.completion: 05-20 22:23:03] {927} INFO - evaluating data instance 2
[flaml.autogen.oai.completion: 05-20 22:23:20] {927} INFO - evaluating data instance 3
[flaml.autogen.oai.completion: 05-20 22:23:47] {927} INFO - evaluating data instance 4
[flaml.autogen.oai.completion: 05-20 22:23:47] {927} INFO - evaluating data instance 5
[flaml.autogen.oai.completion: 05-20 22:23:48] {927} INFO - evaluating data instance 6
[flaml.autogen.oai.completion: 05-20 22:23:48] {927} INFO - evaluating data instance 7
[flaml.autogen.oai.completion: 05-20 22:23:48] {927} INFO - evaluating data instance 8
[flaml.autogen.oai.completion: 05-20 22:24:20] {927} INFO - evaluating data instance 9
[flaml.autogen.oai.completion: 05-20 22:24:20] {927} INFO - evaluating data instance 10
[flaml.autogen.oai.completion: 05-20 22:24

What about the default, untuned gpt-4 config (with the same prompt as the tuned config)? We can evaluate it and compare:

In [16]:
# the following code will cost roughly $2 if uncommented and run.

# default_config = {"model": 'gpt-4', "prompt": prompts[0]}
# default_result = oai.ChatCompletion.test(test_data, config_list=config_list, **default_config)
# print("performance on test data from gpt-4 with a default config:", default_result)

performance on test data from gpt-4 with a default config: {'expected_success': 0.6965174129353234, 'success': 0.6965174129353234, 'success_vote': 0.6965174129353234, 'votes': 1.0, 'cost': 1.9264799999999993, 'inference_cost': 0.009584477611940295}


In [17]:
# print("tuned config succeeds in {:.1f}% test cases".format(result["success_vote"] * 100))
# print("untuned config succeeds in {:.1f}% test cases".format(default_result["success_vote"] * 100))

tuned config succeeds in 90.5% test cases
untuned config succeeds in 69.7% test cases


The default use of GPT-4 has a much lower accuracy. Note that the default config has a lower inference cost. What if we heuristically increase the number of responses n?

In [18]:
# The following evaluation costs $3 and longer than one hour if you uncomment it and run it.

# config_n2 = {"model": 'gpt-4', "prompt": prompts[0], "n": 2}
# result_n2 = oai.ChatCompletion.test(test_data, config_list=config_list, **config_n2)
# print("performance on test data from gpt-4 with a default config and n=2:", result_n2)


The inference cost is doubled and matches the tuned config. But the success rate doesn't improve much. What if we further increase the number of responses n to 5?

In [19]:
# The following evaluation costs $8 and longer than one hour if you uncomment it and run it.

# config_n5 = {"model": 'gpt-4', "prompt": prompts[0], "n": 5}
# result_n5 = oai.ChatCompletion.test(test_data, config_list=config_list, **config_n5)
# print("performance on test data from gpt-4 with a default config and n=5:", result_n5)

We find that the 'success_vote' metric is increased at the cost of exceeding the inference budget. But the tuned configuration has both higher 'success_vote' (91% vs. 87%) and lower average inference cost ($0.015 vs. $0.037 per instance).

A developer could use flaml to tune the configuration to satisfy the target inference budget while maximizing the value out of it.